In [1]:
import pyspark
from pyspark.sql.functions import current_timestamp
from delta import *

In [2]:
def create_spark_session():
    conf = (
        pyspark.conf.SparkConf()
        .setAppName("LetsTalk")
        .set(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
        .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
        .set("spark.hadoop.google.cloud.auth.service.account.enable", "true")
        .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/alfio/projects/upc/LetsTalkBDM/gcs.json")
        .set("spark.sql.shuffle.partitions", "4")
        .set("spark.jars", "../jars/gcs-connector-hadoop3-latest.jar")
        .setMaster(
            "local[*]"
        )
    )

    builder = pyspark.sql.SparkSession.builder.appName("LetsTalk").config(conf=conf)
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    return spark



In [3]:
spark = create_spark_session()

25/04/29 12:25:11 WARN Utils: Your hostname, Alfios-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.50.4 instead (on interface en0)
25/04/29 12:25:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/alfio/.ivy2/cache
The jars for the packages stored in: /Users/alfio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3ba2087b-e7e3-429d-9be8-c85bb527d131;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central


:: loading settings :: url = jar:file:/Users/alfio/python_venv/general/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


:: resolution report :: resolve 65ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-3ba2087b-e7e3-429d-9be8-c85bb527d131
	confs: [default]
	0 artifacts copied, 3 already retrieved (0kB/2ms)
25/04/29 12:25:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log l

In [4]:
deltaTable = DeltaTable.forPath(spark, "../data/letstalk_landing_zone_bdma/delta_news/entertainment")
df = deltaTable.toDF()

In [9]:
df.orderBy('publishedAt').show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              author|             content|         description|         publishedAt|              source|               title|                 url|          urlToImage|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          Lacey Rose|Lalisa Manobal, a...|The cast and crew...|2025-04-03T14:30:00Z|{NULL, Hollywood ...|A Secret, a Secur...|http://www.hollyw...|https://www.holly...|
|                    |A computer game k...|Jack Black and Ja...|2025-04-03T15:10:16Z|         {NULL, NPR}|See 'A Minecraft ...|https://www.npr.o...|https://npr.brigh...|
| Kathryn VanArendonk|Pulse makes clear...|The chief effect ...|2025-04-03T18:00:20Z|     {NULL, Vulture}|‘Pulse’ Review: N...|http://www.vultur...|ht